In [1]:
import os
import getpass
from langchain_mistralai import ChatMistralAI
from langchain_mistralai import MistralAIEmbeddings

if "MISTRAL_API_KEY" not in os.environ:
    os.environ["MISTRAL_API_KEY"] = getpass.getpass("Enter your Mistral API key: ")

model=ChatMistralAI(model="mistral-large-latest",temperature=1)
embeddings = MistralAIEmbeddings()
print(model.invoke("Tell me a joke"))


c:\Users\prakhar.parashar\OneDrive - Blue Altair\Desktop\RAG_solution\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\prakhar.parashar\OneDrive - Blue Altair\Desktop\RAG_solution\.venv\Lib\site-packages\langchain_mistralai\embeddings.py:181: UserWarning: Could not download mistral tokenizer from Huggingface for calculating batch sizes. Set a Huggingface token via the HF_TOKEN environment variable to download the real tokenizer. Falling back to a dummy tokenizer that uses `len()`.
  warnings.warn(


content='What do you call fake spaghetti? An impasta' additional_kwargs={} response_metadata={'token_usage': {'prompt_tokens': 7, 'total_tokens': 21, 'completion_tokens': 14}, 'model_name': 'mistral-large-latest', 'model': 'mistral-large-latest', 'finish_reason': 'stop'} id='run-820c6e8c-a95e-408b-a103-68718317187d-0' usage_metadata={'input_tokens': 7, 'output_tokens': 14, 'total_tokens': 21}


In [2]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()

chain= model | parser
chain.invoke("tell me a joke about space.")

'What do you call a crazy spaceman? An astro-nut'

In [16]:
from langchain_community.document_loaders import PyPDFLoader, TextLoader, Docx2txtLoader

doc_file=input("Enter the file name along with it's extension:")
if doc_file.endswith(".pdf"):
    loader=PyPDFLoader(doc_file)
elif doc_file.endswith(".docx"):
    loader=Docx2txtLoader(doc_file)
elif doc_file.endswith(".txt") or doc_file.endswith(".md"):
    loader=TextLoader(doc_file)  
else: 
    print(f"Document type {doc_file.type} not supported.")          

pages=loader.load_and_split()
pages

[Document(metadata={'source': 'RAG_DevOps.docx'}, page_content="Skip to main content \n\nClick here to return to Amazon Web Services homepage \n\nAbout AWS Contact Us Support \xa0 English \xa0 My Account \xa0 \n\nSign In \n\nCreate an AWS Account \n\nClose \n\nProfile \n\nYour profile helps improve your interactions with select AWS experiences. \n\nLogin \n\nClose \n\nProfile \n\nYour profile helps improve your interactions with select AWS experiences. \n\nView profile \n\nLog out \n\nAmazon Q\n\nProducts\n\nSolutions\n\nPricing\n\nDocumentation\n\nLearn\n\nPartner Network\n\nAWS Marketplace\n\nCustomer Enablement\n\nEvents\n\nExplore More \n\nTop of Form\n\nBottom of Form\n\nClose \n\nعربي\n\nBahasa Indonesia\n\nDeutsch\n\nEnglish\n\nEspañol\n\nFrançais\n\nItaliano\n\nPortuguês\n\nTiếng Việt\n\nTürkçe\n\nΡусский\n\nไทย\n\n日本語\n\n한국어\n\n中文 (简体)\n\n中文 (繁體)\n\nClose \n\nMy Profile\n\nSign out of AWS Builder ID\n\nAWS Management Console\n\nAccount Settings\n\nBilling & Cost Management\n\n

In [8]:
from langchain import PromptTemplate

custom_instruction = """
You are a helpful AI assistant. Answer the questions based on the provided context.
If you don't know the answer, reply with "I don't know."

Context: {context}
Question: {question}
"""

# Create a single prompt template with the instruction included
prompt = PromptTemplate.from_template(custom_instruction)

chain = prompt | model | parser

print(chain.invoke({"context": "Pune is getting hotter day by day", "question": "is pune cool or hot?"}))



Based on the provided context, Pune is getting hotter. So, it is hot.


In [9]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore=DocArrayInMemorySearch.from_documents(pages,embedding=embeddings)

c:\Users\prakhar.parashar\OneDrive - Blue Altair\Desktop\RAG_solution\.venv\Lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [ ]:
retriever=vectorstore.as_retriever()
# retriever.invoke("Andrew Tate")

[Document(metadata={'source': 'BA - DevOps Assignment - Prakhar Parashar.docx'}, page_content='DEVOPS ASSIGNMENT\n\nTask:\n\nYou are tasked with automating the build & deployment of a website.\n\nUse any free hosting service (GitHub Pages, Netlify, or Vercel).\n\n\n\nPre-requisites:\n\nA website to deploy\n\n\n\n\n\n\n\n\n\nStep-by-step instructions:\n\nLog in to GitHub and create a new repository where we will store the static website\n\nOpen VS Code and in the terminal Use the “git add .” command to get all new and modified files for commit. This makes sure we have all files.\n\n\n\n\n\nBefore Committing we have to set up the profile using:\ngit config --global user.email  “parasharprakhar.1123@gmail.com”\ngit config --global user.name "Prakhar11235" ,\n\n\n\nCommit the changes with an understandable message.\n\n\n\nPush the files to repository using “git push origin main”\n\n\n\nGenerate a Vercel Access Token and Configure GitHub Secrets\n\n    Create an access token in Vercel by na

In [15]:
from operator import itemgetter

chain=(
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question")
    }
    | prompt
    | model
    |parser
)

chain.invoke({"question":"what are vercel access tokens?"})

'Vercel access tokens are security credentials that allow you to connect your GitHub repository to Vercel for continuous deployment. These tokens act as a form of authentication, enabling Vercel to access and deploy updates from your repository automatically. In the context provided, you create an access token in Vercel by navigating to Settings > Tokens, and then you use this token to configure a GitHub secret named `VERCEL_TOKEN`. This process ensures that any changes pushed to your GitHub repository trigger an automated deployment to Vercel.'